The following code is an example of a data cleansing process that using regex to extract differently formatted dates from records in a text file. It locates the dates in the string, formats them and stores them in the 'formatted_dates' field. This is done in a single pass over the dataset. Examples of date formats that might occur anywhre in a string follow:

* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

In [110]:
import pandas as pd
import re
from datetime import datetime 

In [111]:
def convert_list_to_df(doc):
    """
    Given a list convert it to DataFrame containing the following columns:
    * 'Text'
    * 'formatted_date'.
    """
    df = pd.DataFrame(doc, columns=['Text'])
    df['formatted_date'] = pd.Series()
    return df

In [112]:
def date_sorter():
    result = pd.Series(df.index)
    return result

In [113]:
def date_transformer(df):
    """
    Applies a lambda to each row in the Dataframe. Each record in the dataset is processed
    once.
    """
    df = df.apply(lambda row: process_text(row), axis=1)
    return df

In [114]:
def get_day(day):
    """
    Returns day of month in 'dd' format.
    """
    day = day.replace(',','').replace('.', '')
    if len(day) == 2:
        return day
    else:
        return '0' + day

In [115]:
def get_month(month):
    """
    Returns string with month in 'mm' format.
    """
    month = month.replace(',', '').replace('.', '')
    if len(month) == 2:
        return month
    else:
        return '0' + month

In [116]:
def get_month_from_name(month):
    """ 
    Given an alpha description of a month a string containing the numeric 
    equivalent is returned.
    """
    
    result = re.match(r'[Jj]an[a-z,.]*', month)
    if result is not None:
        return '01'
    result = re.match(r'[Ff]eb[a-z,.]*', month)
    if result is not None:
        return '02'
    result = re.match(r'[Mm]ar[a-z,.]*', month)
    if result is not None:
        return '03'
    result = re.match(r'[Aa]pr[a-z,.]*', month)
    if result is not None:
        return '04'
    result = re.match(r'[Mm]ay[a-z,.]*', month)
    if result is not None:
        return '05'
    result = re.match(r'[Jj]un[a-z,.]*', month)
    if result is not None:
        return '06'
    result = re.match(r'[Jj]ul[a-z,.]*', month)
    if result is not None:
        return '07'
    result = re.match(r'[Aa]ug[a-z,.]*', month)
    if result is not None:
        return '08'
    result = re.match(r'[Ss]ep[a-z,.]*', month)
    if result is not None:
        return '09'
    result = re.match(r'[Oo]ct[a-z,.]*', month)
    if result is not None:
        return '10'
    result = re.match(r'[Nn]ov[a-z,.]*', month)
    if result is not None:
        return '11'
    result = re.match(r'[Dd]ec[a-z,.]*', month)
    if result is not None:
        return '12'
    
    return None

In [117]:
def get_year(year):
    """
    Returns year in 'yyyy' format.
    """
    if len(year) == 4:
        return year
    else:
        return '19' + year

In [118]:
def extract_data():
    """
    Extracts data from 'dates.txt' file. This file consists of clinic patient notes
    that contain dates that are in different formats.
    """
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)
    return doc

In [119]:
def process_text(row):
    """
    Extracts the date contained in the 'Text' field and stored a formatted date in the
    'formatted_date' field. Where applicable the method processes data from most specific
    to least specific.
    """
    # Check for date in the format similar to mm/dd/yyyy.
    tmp = re.search(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', row['Text'])
    if tmp is not None:
        try:
            value = tmp.group()
            if '-' in value:
                mdy = tmp.group().split('-')
            else:
                mdy = tmp.group().split('/')
            mm = mdy[0]
            if len(mdy[0]) == 1:
                mm = '0' + mdy[0]
            dd = mdy[1]
            if len(mdy[1]) == 1:
                dd = '0' + mdy[1]
            yyyy = mdy[2]
            if len(mdy[2]) == 2:
                yyyy = '19' + mdy[2]
            row['formatted_date'] = get_date_object('{}/{}/{}'.format(mm, dd, yyyy))
        except:
            print('ERROR_0 > {} -- {}'.format(tmp.group(), row['Text']))
        return row
    
    # Check for date in 'month year' format. For example, 'Feb 2009', 'June 97'.
    tmp = re.search(r'(\d{1,2} ([Jj]an|[Ff]eb|[Mm]ar|[Aa]pr|[Mm]ay|[Jj]un|[Jj]ul|[Aa]ug|'
                    r'[Ss]ep|[Oo]ct|[Nn]ov|[Dd]ec)[a-z,.]* \d{2,4})', row['Text'])  
    if tmp is not None:
        try:
            dmy = tmp.group().split(' ')
            dd = get_day(dmy[0].strip())
            mm = get_month_from_name(dmy[1].strip())
            yyyy = get_year(dmy[2].strip())
            row['formatted_date'] = get_date_object('{}/{}/{}'.format(mm, dd, yyyy))
        except:
            print('ERROR_1 > {} -- {}'.format(tmp.group(), row['Text']))
        return row
    
    # Check for date in 'month dd, yyyy'. For example, Jan. 20, 2015.
    tmp = re.search(r'(([Jj]an|[Ff]eb|[Mm]ar|[Aa]pr|[Mm]ay|[Jj]un|[Jj]ul|[Aa]ug|[Ss]ep|'
                    r'[Oo]ct|[Nn]ov|[Dd]ec)[a-z,.]*[\s.-]+\d{1,2}[\s,-]+\d{2,4})', row['Text'])
    if tmp is not None:
        try:
            value = tmp.group()
            if '-' in value:
                mdy = tmp.group().split('-')
            else:
                mdy = tmp.group().split(' ')
            dd = get_day(mdy[1].strip())
            mm = get_month_from_name(mdy[0].strip())
            yyyy = get_year(mdy[2].strip())
            row['formatted_date'] = get_date_object('{}/{}/{}'.format(mm, dd, yyyy))
        except:
            print("ERROR_3 > ", tmp.group())
        return row
    
    # Find date similar to June, year. This will be stored as 'mm/01/year'.
    tmp = re.search(r'(([Jj]an|[Ff]eb|[Mm]ar|[Aa]pr|[Mm]ay|[Jj]un|[Jj]ul|[Aa]ug|[Ss]ep|'
                    r'[Oo]ct|[Nn]ov|[Dd]ec)[a-z,.]* \d{4})', row['Text'])
    if tmp is not None:
        try:
            my = tmp.group().split(' ')
            mm = get_month_from_name(my[0].strip())
            yyyy = get_year(my[1].strip())
            row['formatted_date'] = get_date_object('{}/01/{}'.format(mm, yyyy))
        except:
            print("ERROR_4 > ", tmp.group())
        return row
    
    # Find date similar to 'month/year' format. This will be stored as mm/01/yyyy.
    tmp = re.search(r'(^| )(\d{1,2}[/]\d{2,4})', row['Text'])
    if tmp is not None:
        try:
            my = tmp.group().split('/')
            mm = get_month(my[0].strip())
            yyyy = get_year(my[1].strip())
            row['formatted_date'] = get_date_object('{}/01/{}'.format(mm, yyyy))
        except:
            print("ERROR_5 > ", tmp.group())
        return row
    
    # Find a year. This will be stored as: '01/01/1998'
    tmp = re.search(r'(^| )(\d{4})', row['Text'])
    if tmp is not None:
        try:
            yyyy = tmp.group().strip()
            row['formatted_date'] = get_date_object('01/01/{}'.format(get_year(yyyy)))
        except:
            print("ERROR_6 > ", tmp.group())
        return row
    
    # Find a year. This will be stored as: '01/01/1997'.
    tmp = re.search(r'(\d{4})', row['Text'])
    if tmp is not None:
        try:
            yyyy = tmp.group().strip()
            row['formatted_date'] = get_date_object('01/01/{}'.format(get_year(yyyy)))
        except:
            print("ERROR_7 > ", tmp.group())
        return row
    
    # Unable to find a matching date format.
    row['formstted_date'] = set_date('01/01/1900')
    return row

In [120]:
def get_date_object(str_date):
    """ 
    Given a date in 'mm/dd/yyyy' format returns the associated date object.
    """
    return datetime.strptime(str_date, '%m/%d/%Y')

In [121]:
# Load data.
doc = []
doc = extract_data()

# Convert to pandas dataframe.
df = convert_list_to_df(doc)

# Read Text field to extract date and place in 'formatted_date' field.
df = date_transformer(df)
df = df.sort_values(by=['formatted_date'])

# Print samples.
print(df.head())
print(df.tail())

                                                 Text formatted_date
9            (4/10/71)Score-1Audit C Score Current:\n     1971-04-10
84        5/18/71 Total time of visit (in minutes):\n     1971-05-18
2   sshe plans to move as of 7/8/71 In-Home Servic...     1971-07-08
53                      7/11/71 SOS-10 Total Score:\n     1971-07-11
28                             9/12/71 [report_end]\n     1971-09-12
                                                  Text formatted_date
231  50 yo DWF with a history of alcohol use disord...     2016-05-01
141                  30 May 2016 SOS-10 Total Score:\n     2016-05-30
186                 13 Oct 2016 Primary Care Doctor:\n     2016-10-13
161  19 Oct 2016 Communication with referring physi...     2016-10-19
413                     11/2016 Primary Care Doctor:\n     2016-11-01
